# CX2Network: Access node, edge, and network attributes

In [2]:
from ndex2.cx2 import CX2Network, RawCX2NetworkFactory
from ndex2 import constants
import ndex2.client as nc
import ndex2
import networkx as nx
import pandas as pd
import os
import json

In [3]:
# EXAMPLE NETWORK FROM NDEx

uuid='f1dd6cc3-0007-11e6-b550-06603eb7f303'

# Create NDEx2 python client
client = ndex2.client.Ndex2(host='ndexbio.org')

# Create CX2Network factory
factory = RawCX2NetworkFactory()

# Get network as cx2 stream
client_resp = client.get_network_as_cx2_stream(uuid)

# Convert downloaded network to CX2Network object
cx2_network = factory.get_cx2network(json.loads(client_resp.content))

### Nodes

Loop through all **nodes** and print when we reach the RAD52 node

In [4]:
for node_id, node in cx2_network.get_nodes().items():
    if node.get('v')['name'] == 'RAD52':
        print(node)
        break

{'id': 1114201, 'v': {'name': 'RAD52', 'entrezgene': 5893, 'Yeast (systematic)': 'YML032C', 'EnrichedFunction2': '', 'Category': 'TSG', 'Pathway': 'DNA Damage Control', 'Color': '', 'mut/length ratio': 0.012, 'Mut_count': 31, 'Yeast (standard)': 'RAD52', 'drug status': 'n/a', 'FunctionLabel': 'DNA recombination', 'drug': 'n/a', 'AltFnLabel': 'Homologous recombination', 'ProcessList': 'double-strand break repair via homologous recombination|DNA recombinase assembly|DNA metabolic process|DNA repair|double-strand break repair|DNA recombination|response to stress|biological_process|cellular response to oxidative stress', 'Functional Categor': 'Genome Maintenance', 'molecular function': '', 'Synonym': 'E9PF95|RAD52|E9PF95_HUMAN', 'biological process': 'DNA recombinase assembly|double-strand break repair via single-strand annealing', 'Mutation Frequency': 0.0067, 'DB_Object_Name': 'DNA repair protein RAD52 homolog'}, 'x': -516.2893336879745, 'y': -15.998864721426372, 'z': None}


Nodes and edges are indexed by id (@id from the example above).  To look up a specific node by it's _name_ you can create a reverse look up index as follows

In [5]:
#============================
# CREATE A NAME to ID LOOKUP 
#============================
node_name_lookup = {node.get('v').get('name', None): node_id for node_id, node in cx2_network.get_nodes().items()}

#========================
# GET THE 'MAP2K1' NODE
#========================
map2k1_node_id = node_name_lookup.get('MAP2K1')
map2k1_node = cx2_network.get_node( map2k1_node_id )
print(map2k1_node)

#========================================================
# GET THE 'MAP2K1' NODE (USING METHOD FROM CX2NETWORK)
#========================================================
map2k1_node_id = cx2_network.lookup_node_id_by_name( 'MAP2K1' )
map2k1_node = cx2_network.get_node(map2k1_node_id)

{'id': 1114200, 'v': {'name': 'MAP2K1', 'entrezgene': 5604, 'Yeast (systematic)': 'YJL128C', 'EnrichedFunction2': '', 'Category': 'druggable,TSG', 'Pathway': 'Signal Transduction / Growth Regulation', 'Color': '1.0', 'mut/length ratio': 0.024, 'Mut_count': 61, 'Yeast (standard)': 'PBS2', 'drug status': 'Phase 2', 'FunctionLabel': 'cell cycle', 'drug': 'PD0325901', 'AltFnLabel': 'MAPK signaling', 'ProcessList': 'MAPK cascade|activation of MAPKK activity|cell morphogenesis|regulation of protein phosphorylation|cellular protein modification process|protein phosphorylation|cellular component movement|response to stress|response to oxidative stress|cell cycle|cell cycle arrest|mitotic nuclear division|signal transduction|small GTPase mediated signal transduction|Ras protein signal transduction|biological_process|pathogenesis|positive regulation of gene expression|peptidyl-tyrosine phosphorylation|regulation of stress-activated MAPK cascade|positive regulation of transcription elongation fro

### Node attributes

Get the node attribute (Pathway) for MAP2K1

In [6]:
map2k1_pathway_attribute = cx2_network.get_node(map2k1_node_id).get(constants.ASPECT_VALUES).get('Pathway')
print(map2k1_pathway_attribute)

Signal Transduction / Growth Regulation


### Edges

Print all the **edges** where MAP2K1 is either a source or target

In [7]:
map2k1_edges = []

for edge_id, edge in cx2_network.get_edges().items():
    if edge.get('s') == map2k1_node_id or edge.get('t') == map2k1_node_id:
        map2k1_edges.append(edge.get('id'))

        print(edge)


{'id': 1114251, 's': 1114200, 't': 1114208, 'v': {'Product': 2.660014701, 'yeastSscore': -4.896, 'T-stat': -3.025, 'yeast2': 'YBL016W', 'HumanRanks': 1486, 'yeast1': 'YJL128C', 'name': 'MAP2K1 (pp) MAPK1', 'druggable': 'MAP2K1', 'interaction': 'pp', 'prior report of SL': 'no', 'YeastRanks': 1692, 'humanFDRnegLog10': 0.879, 'ProductRanks': 0.847916085, 'TS': 'MAPK1'}}
{'id': 1114250, 's': 1114200, 't': 1114201, 'v': {'Product': 3.495852536, 'yeastSscore': -4.649, 'T-stat': -3.349, 'yeast2': 'YML032C', 'HumanRanks': 1521, 'yeast1': 'YJL128C', 'name': 'MAP2K1 (pp) RAD52', 'druggable': 'MAP2K1', 'interaction': 'pp', 'prior report of SL': 'no', 'YeastRanks': 1688, 'humanFDRnegLog10': 1.044, 'ProductRanks': 0.865835448, 'TS': 'RAD52'}}


Or, if you would like to see the node labels represented in these edges you can look up and print the node name

In [8]:
for edge_id, edge in cx2_network.get_edges().items():
    if edge.get('s') == map2k1_node_id or edge.get('t') == map2k1_node_id:
        source_node = cx2_network.get_node( edge.get('s') )
        target_node = cx2_network.get_node( edge.get('t') )
        print('Source: %s Target: %s Interaction: %s' % (source_node.get(constants.ASPECT_VALUES).get('name'), target_node.get(constants.ASPECT_VALUES).get('name'), edge.get(constants.ASPECT_VALUES).get('interaction')))

Source: MAP2K1 Target: MAPK1 Interaction: pp
Source: MAP2K1 Target: RAD52 Interaction: pp


### Edge attributes

Print the **edge attributes** (yeastSscore) for MAP2K1 edges

In [9]:
for edge_id in map2k1_edges:
    print(cx2_network.get_edge(edge_id).get(constants.ASPECT_VALUES).get('yeastSscore'))


-4.896
-4.649


### Network attributes

Print the reference attribute for this network

In [10]:
cx2_network.get_network_attributes().get('reference')

'<div style="font-size: 12px;"><i><span style="float: none;">Srivas R, Shen JP, Yang CC, Sun SM, Li J,</span><br/></i><span style="float: none;"><i>Gross AM, Jensen J, Licon K, Bojorquez-Gomez A, Klepper K, Huang J, Pekin D, Xu\xa0JL, Yeerna H, Sivaganesh V, Kollenstart L, van Attikum H, Aza-Blanc P, Sobol\xa0RW, Ideker T</i>.\xa0</span><b><b>A network of conserved synthetic lethal interactions for exploration\xa0</b></b><b><b>of precision cancer therapy</b></b><span style="float: none;">.</span></div><div style="font-size: 12px;"><span style="float: none;">Mol Cell. 2016 Aug 4;63(3):514-25.</span><br/><a href="http://doi.org/10.1016/j.molcel.2016.06.022" target="">doi: 10.1016/j.molcel.2016.06.022</a></div>'

Or print the html content in the value field

In [11]:
from IPython.core.display import display, HTML

display(HTML(cx2_network.get_network_attributes().get('reference')))

/var/folders/gx/bx0pj8d56_1920zjml13xtq00000gq/T/ipykernel_68259/4233257348.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML
